# 중국 대기질 데이터 크롤링

- 대상 사이트 : http://www.aqistudy.cn/historydata/
- 참고 소스코드 : https://github.com/Nyloner/Nyspider/blob/master/www.aqistudy.cn/aqistudy.py
- 수정 날짜 : 2019/10/23

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common import exceptions
import time
import logging
import requests
import time
import datetime as dt
import urllib.parse
import os
import pandas as pd 

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

driver = webdriver.Chrome('./chromedriver', chrome_options=options)

In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import os
import time
import re

headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:39.0) Gecko/20100101 Firefox/39.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate'}

def get_city():
    html=requests.get('http://www.aqistudy.cn/historydata/',headers=headers).text
    table=BeautifulSoup(html,'lxml').find('div',{'class':'all'}).find_all('a')
    cities=[]
    for item in table:
        try:
            name=item.get_text()
            url='http://www.aqistudy.cn/historydata/'+item.get('href')
            cities.append({'name':name,'url':url})
        except:
            continue
    
    return cities

def get_data(url):
    count=0
    
    while True:
        try:
            #print(url)
            driver.get(url)
            time.sleep(3)
            html = driver.page_source
            break
        except:
            count+=1
            if count==5:
                return []
            
    #print(html)
    table= BeautifulSoup(html,'lxml').select("body > div.container > div.row > div.col-lg-9.col-md-8.col-sm-8.col-xs-12 tr")
    #print(table)
    
    result=[]
    for item in table[1:]:
        try:
            tds=item.find_all('td')
            line=''
            for td in tds:
                line +=td.get_text().replace('\r','').replace('\n','').replace('\t','') +','
            result.append(line)
        except:
            continue
    #print(result)
    return result


def crawler(item):
    try:
        exists=os.listdir('result/'+item['name'])
    except:
        exists=[]
    try:
        os.mkdir('result')
    except:
        pass
    try:
        os.mkdir('result/'+item['name'])
    except:
        pass
    count=0
    #print(item)
    while True:
        try:
            html=requests.get(item['url'],headers=headers).text
            break
        except:
            count+=1
            if count==5:
                return
    
    bs=BeautifulSoup(html,'lxml')
    table = bs.select("body > div.container > div.row > div.col-lg-3.col-md-4.col-sm-4.col-xs-12 > div:nth-of-type(2) > div:nth-of-type(2) > ul > li > a")
    
    dates = []
    for tr in table:
        fn = re.findall("\d+",tr.text)
        fn = fn[0] + "-" + fn[1]
        try:
            if fn +'.txt' in exists:
                continue
            dates.append(fn)
        except:
            continue
    
    #print(dates)
    for date in dates:
        try:
            data= get_data('https://www.aqistudy.cn/historydata/daydata.php?city=%s&month=%s'%(item['name'],date))
            print(data)
        except:
            continue
        
        if data==[] or data == [''] :
            continue
        
        fn = ('./result/%s/%s.txt'%(item['name'],date))
        print(len(data))
        result_df = pd.DataFrame(data)
        result_df = result_df[0]
        result_df.to_csv(fn,index=False)

def main():
    cities = get_city()
    for city in cities:
        try:
            #print(city)
            crawler(city)
        except:
            print(city,'failed')
    print('End')

main()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['2014-12-31,53,良,33,55,3,1,23,69,']
1
['2015-01-01,38,优,26,38,3,0.6,13,38,', '2015-01-02,34,优,24,34,3,0.6,14,40,', '2015-01-03,38,优,25,38,4,0.6,17,39,', '2015-01-04,36,优,22,36,6,0.8,22,37,', '2015-01-05,33,优,21,33,6,0.7,14,51,', '2015-01-06,36,优,25,34,5,0.6,10,69,', '2015-01-07,35,优,20,27,2,0.5,14,69,', '2015-01-08,35,优,22,22,2,0.5,10,69,', '2015-01-09,35,优,21,26,2,0.5,10,69,', '2015-01-10,35,优,22,22,2,0.5,10,70,', '2015-01-11,28,优,10,22,2,0.4,10,55,', '2015-01-12,26,优,7,20,2,0.5,8,52,', '2015-01-13,29,优,21,26,2,0.5,14,54,', '2015-01-14,34,优,24,25,4,0.6,16,52,', '2015-01-15,28,优,19,25,5,0.6,16,55,', '2015-01-16,36,优,25,28,6,0.6,18,63,', '2015-01-17,32,优,17,20,4,0.5,11,64,', '2015-01-18,31,优,18,25,4,0.4,14,62,', '2015-01-19,34,优,22,21,4,0.5,15,68,', '2015-01-20,30,优,19,25,4,0.4,12,60,', '2015-01-21,27,优,19,27,4,0.5,10,54,', '2015-01-22,24,优,15,24,3,0.5,12,46,', '2015-01-23,27,优,16,27,4,0.6,21,44,', '2015-01-24,26,优,18,25,5,0.5,14,46,', '2015-01-25,29

['2015-09-01,45,优,31,44,4,0.6,9,56,', '2015-09-02,42,优,29,42,4,0.6,8,42,', '2015-09-03,45,优,31,36,4,0.6,5,43,', '2015-09-05,45,优,31,37,7,0.5,6,70,', '2015-09-06,35,优,24,35,4,0.4,3,62,', '2015-09-07,46,优,32,42,3,0.5,6,46,', '2015-09-08,50,优,33,50,6,0.6,5,37,', '2015-09-09,50,优,35,47,19,0.6,3,56,', '2015-09-10,53,良,37,48,3,0.6,4,52,', '2015-09-11,52,良,36,41,4,0.5,5,39,', '2015-09-13,50,优,35,47,6,0.6,5,44,', '2015-09-14,42,优,23,42,5,0.7,5,52,', '2015-09-15,33,优,13,33,5,0.4,5,66,', '2015-09-17,29,优,17,29,3,0.5,6,54,', '2015-09-18,31,优,16,22,5,0.5,5,62,', '2015-09-19,36,优,15,28,5,0.5,6,71,', '2015-09-21,39,优,24,39,5,0.9,6,62,', '2015-09-22,39,优,21,39,3,0.4,8,78,', '2015-09-23,43,优,24,43,5,0.4,6,51,', '2015-09-24,42,优,29,39,6,0.4,6,55,', '2015-09-27,52,良,31,54,7,0.5,7,50,', '2015-09-28,53,良,28,55,4,0.3,7,43,', '2015-09-29,51,良,29,51,6,0.4,9,50,', '2015-09-30,47,优,24,47,6,0.5,8,68,']
24
['2015-10-01,78,良,36,106,5,0.5,10,88,', '2015-10-02,38,优,19,38,4,0.4,5,72,', '2015-10-03,37,优,19,30,5,0.3,5

['2016-05-01,70,良,16,34,8,0.5,12,124,', '2016-05-02,75,良,16,40,7,0.8,8,130,', '2016-05-03,74,良,22,57,20,0.8,12,128,', '2016-05-04,71,良,22,48,7,0.5,12,125,', '2016-05-05,74,良,28,50,10,0.7,16,128,', '2016-05-06,64,良,28,48,8,0.8,15,116,', '2016-05-07,49,优,19,25,9,0.4,18,97,', '2016-05-08,58,良,22,24,4,0.5,13,109,', '2016-05-09,65,良,19,45,6,0.9,10,118,', '2016-05-10,52,良,19,34,8,1,12,102,', '2016-05-11,62,良,18,32,3,1,14,114,', '2016-05-12,60,良,16,34,6,1,12,112,', '2016-05-13,82,良,18,30,2,1.3,12,138,', '2016-05-14,64,良,18,26,6,1.1,10,116,', '2016-05-15,59,良,12,20,5,0.7,11,110,', '2016-05-16,47,优,14,28,7,0.7,8,93,', '2016-05-17,51,良,10,24,8,0.7,14,101,', '2016-05-18,44,优,13,37,5,0.6,12,88,', '2016-05-19,47,优,14,34,10,0.5,10,94,', '2016-05-20,69,良,15,42,12,0.3,8,122,', '2016-05-21,55,良,14,32,7,0.3,8,106,', '2016-05-22,46,优,12,22,6,0.4,10,92,', '2016-05-23,61,良,11,28,5,0.3,8,113,', '2016-05-24,49,优,13,36,6,0.4,10,98,', '2016-05-25,46,优,16,36,4,0.7,10,92,', '2016-05-26,44,优,11,19,4,0.7,7,88,', '

['2016-12-01,42,优,16,32,14,0.4,12,84,', '2016-12-02,32,优,14,24,12,0.6,13,63,', '2016-12-03,46,优,18,36,15,0.5,13,91,', '2016-12-04,42,优,18,42,14,0.4,18,70,', '2016-12-05,52,良,27,53,16,0.6,22,62,', '2016-12-06,47,优,24,47,20,0.7,22,66,', '2016-12-07,46,优,24,46,15,0.6,18,68,', '2016-12-08,53,良,23,56,15,0.8,24,64,', '2016-12-09,48,优,28,48,16,0.8,22,58,', '2016-12-10,53,良,32,56,17,0.8,23,50,', '2016-12-11,57,良,37,64,15,0.8,24,48,', '2016-12-12,67,良,44,84,15,0.8,26,49,', '2016-12-13,78,良,34,106,15,1,30,48,', '2016-12-14,57,良,35,64,14,0.8,22,59,', '2016-12-15,55,良,35,60,15,1,24,55,', '2016-12-16,51,良,24,52,16,0.8,21,62,', '2016-12-17,53,良,30,56,16,0.8,22,54,', '2016-12-18,55,良,26,60,18,0.7,21,57,', '2016-12-19,61,良,30,71,18,0.8,24,50,', '2016-12-20,55,良,29,60,18,0.8,20,50,', '2016-12-21,52,良,26,54,14,0.6,20,72,', '2016-12-22,55,良,28,60,14,0.8,23,44,', '2016-12-23,54,良,26,58,17,0.8,25,49,', '2016-12-24,57,良,32,64,14,0.9,23,60,', '2016-12-25,51,良,26,52,18,0.8,22,60,', '2016-12-26,42,优,26,42,14,0

['2017-08-01,46,优,8,20,8,0.6,9,92,', '2017-08-02,38,优,4,8,7,0.6,8,76,', '2017-08-03,34,优,4,6,8,0.5,8,69,', '2017-08-04,36,优,6,10,7,0.5,6,72,', '2017-08-05,55,良,4,12,6,0.5,8,107,', '2017-08-06,62,良,6,10,8,0.5,10,115,', '2017-08-07,32,优,6,8,8,0.6,10,65,', '2017-08-08,45,优,8,23,6,0.5,7,90,', '2017-08-09,55,良,8,33,8,0.4,8,107,', '2017-08-10,41,优,7,24,6,0.5,8,82,', '2017-08-11,37,优,4,13,7,0.5,8,74,', '2017-08-12,40,优,5,7,6,0.5,6,81,', '2017-08-13,49,优,5,10,6,0.5,6,99,', '2017-08-14,31,优,4,12,7,0.5,8,62,', '2017-08-15,36,优,4,8,8,0.6,6,72,', '2017-08-16,33,优,4,13,7,0.5,8,67,', '2017-08-17,38,优,4,11,6,0.5,7,76,', '2017-08-18,36,优,6,12,7,0.6,10,72,', '2017-08-19,29,优,6,13,8,0.6,8,59,', '2017-08-20,26,优,4,9,8,0.6,8,53,', '2017-08-21,31,优,4,9,8,0.5,7,63,', '2017-08-22,34,优,4,12,8,0.6,8,69,', '2017-08-23,67,良,6,15,8,0.6,10,121,', '2017-08-24,50,优,4,11,8,0.6,10,100,', '2017-08-25,33,优,6,12,8,0.6,9,67,', '2017-08-26,30,优,4,9,8,0.5,8,61,', '2017-08-27,31,优,4,12,8,0.6,7,63,', '2017-08-28,29,优,5,10,8,0

...

['2017-03-01,58,良,29,66,24,1.1,22,100,', '2017-03-02,55,良,32,62,26,1.1,26,96,', '2017-03-03,66,良,40,84,32,1.2,35,101,', '2017-03-04,67,良,47,86,22,1.2,35,80,', '2017-03-05,67,良,48,84,20,1.2,36,66,', '2017-03-06,70,良,44,76,18,1.1,28,124,', '2017-03-07,61,良,38,70,18,1.1,29,114,', '2017-03-08,60,良,36,64,16,1.1,29,112,', '2017-03-09,61,良,44,67,22,1.1,28,98,', '2017-03-10,71,良,52,75,20,1.1,22,121,', '2017-03-11,75,良,52,76,24,1.1,23,131,', '2017-03-12,75,良,54,52,12,1.1,18,124,', '2017-03-13,59,良,10,12,8,0.9,9,111,', '2017-03-14,43,优,16,22,8,0.7,10,87,', '2017-03-15,49,优,27,29,12,0.8,16,98,', '2017-03-16,44,优,30,40,12,1,20,88,', '2017-03-17,56,良,40,62,22,1.2,36,71,', '2017-03-18,53,良,38,52,14,1.1,20,73,', '2017-03-19,78,良,58,66,10,1.2,32,90,', '2017-03-20,53,良,38,47,16,1.2,26,102,', '2017-03-21,61,良,43,52,16,1.2,21,103,', '2017-03-22,64,良,31,33,14,0.8,16,117,', '2017-03-23,45,优,17,30,8,0.9,18,90,', '2017-03-24,44,优,20,44,16,1,25,79,', '2017-03-25,63,良,26,69,16,0.9,24,116,', '2017-03-26,66,良,32

['2017-10-01,27,优,10,18,5,0.9,22,34,', '2017-10-02,32,优,20,33,8,1.2,24,19,', '2017-10-03,34,优,8,14,4,0.7,14,68,', '2017-10-04,34,优,2,6,4,0.7,10,69,', '2017-10-05,28,优,4,13,5,0.7,10,57,', '2017-10-06,21,优,12,20,5,0.6,12,42,', '2017-10-07,30,优,17,30,6,0.8,20,40,', '2017-10-08,35,优,17,39,8,0.9,17,45,', '2017-10-09,28,优,13,24,8,0.8,16,40,', '2017-10-10,23,优,4,8,4,0.5,13,46,', '2017-10-11,20,优,5,7,5,0.6,16,33,', '2017-10-12,30,优,6,14,6,0.8,16,60,', '2017-10-13,25,优,15,22,8,0.8,16,38,', '2017-10-14,35,优,15,20,6,0.7,15,70,', '2017-10-15,44,优,20,27,7,0.7,18,88,', '2017-10-16,41,优,14,22,6,0.6,14,82,', '2017-10-17,45,优,20,22,6,0.7,20,90,', '2017-10-18,31,优,13,20,6,1,21,62,', '2017-10-19,33,优,20,28,7,1,19,61,', '2017-10-20,48,优,26,57,10,0.9,20,55,', '2017-10-21,50,优,31,44,10,0.9,23,36,', '2017-10-22,61,良,42,62,10,1,23,9,', '2017-10-23,50,优,31,48,10,0.9,21,25,', '2017-10-24,75,良,54,70,7,0.9,19,80,', '2017-10-25,77,良,56,74,8,1,34,44,', '2017-10-26,58,良,42,59,8,1,21,68,', '2017-10-27,50,优,35,51,8,0.

['2018-05-01,60,良,30,67,10,0.6,16,111,', '2018-05-02,78,良,28,76,10,0.6,16,133,', '2018-05-03,77,良,34,75,10,0.6,16,132,', '2018-05-04,56,良,36,62,11,0.6,16,86,', '2018-05-05,34,优,18,34,9,0.6,12,49,', '2018-05-06,44,优,18,44,4,0.4,14,72,', '2018-05-07,49,优,14,32,6,0.6,10,98,', '2018-05-08,64,良,18,47,10,0.6,12,116,', '2018-05-09,60,良,27,50,10,0.6,14,112,', '2018-05-10,52,良,36,50,10,0.5,16,98,', '2018-05-11,38,优,24,38,8,0.5,14,58,', '2018-05-12,47,优,22,47,10,0.6,14,92,', '2018-05-13,77,良,28,66,8,0.5,17,132,', '2018-05-14,72,良,36,74,10,0.5,15,126,', '2018-05-15,72,良,34,56,8,0.4,12,126,', '2018-05-16,85,良,38,70,8,0.6,15,142,', '2018-05-17,74,良,46,98,9,0.6,17,109,', '2018-05-18,56,良,36,62,6,0.4,14,78,', '2018-05-19,33,优,16,32,6,0.6,11,65,', '2018-05-20,38,优,6,15,6,0.6,8,76,', '2018-05-21,27,优,6,14,6,0.6,12,54,', '2018-05-22,53,良,16,56,5,0.4,8,84,', '2018-05-23,65,良,32,79,8,0.6,15,81,', '2018-05-24,74,良,48,98,12,0.6,28,67,', '2018-05-25,42,优,24,42,4,0.4,18,56,', '2018-05-26,65,良,34,80,6,0.4,16,5

['2018-12-01,107,轻度污染,80,114,25,1.3,44,9,', '2018-12-02,92,良,68,99,24,1.5,40,36,', '2018-12-03,0,无,0,0,11,0.9,34,18,', '2018-12-04,0,无,0,0,20,0.5,22,48,', '2018-12-05,0,无,0,0,20,0.7,26,45,', '2018-12-06,91,良,51,132,20,0.7,26,42,', '2018-12-07,57,良,40,60,16,0.6,18,50,', '2018-12-08,57,良,40,48,15,0.6,20,43,', '2018-12-09,64,良,46,65,20,0.6,21,42,', '2018-12-10,89,良,66,76,20,0.7,30,30,', '2018-12-11,98,良,73,100,24,0.9,39,44,', '2018-12-12,82,良,60,98,16,1,44,58,', '2018-12-13,92,良,68,107,16,1,46,46,', '2018-12-14,115,轻度污染,87,124,28,1.3,56,40,', '2018-12-15,163,中度污染,124,158,24,1.5,66,16,', '2018-12-16,133,轻度污染,101,140,24,1.5,54,28,', '2018-12-17,97,良,72,112,18,1.3,50,36,', '2018-12-18,120,轻度污染,91,138,27,1.7,52,28,', '2018-12-19,148,轻度污染,113,157,45,1.9,60,11,', '2018-12-20,109,轻度污染,82,134,24,1.6,48,21,', '2018-12-21,144,轻度污染,110,153,26,2,54,10,', '2018-12-22,79,良,58,62,12,1,38,41,', '2018-12-23,67,良,48,67,18,1,31,48,', '2018-12-24,88,良,65,88,20,1,38,39,', '2018-12-25,125,轻度污染,95,106,21,1.3,46

['2019-07-01,74,良,19,50,13,0.6,16,128,', '2019-07-02,75,良,22,59,16,0.5,20,130,', '2019-07-03,61,良,32,72,18,0.6,22,91,', '2019-07-04,40,优,24,40,8,0.4,17,54,', '2019-07-05,51,良,15,34,8,0.4,12,101,', '2019-07-06,52,良,20,52,9,0.6,17,102,', '2019-07-07,70,良,17,48,14,0.4,16,124,', '2019-07-08,48,优,20,34,14,0.6,16,96,', '2019-07-09,74,良,20,38,9,0.6,14,128,', '2019-07-10,74,良,16,39,10,0.6,17,128,', '2019-07-11,48,优,24,48,8,0.6,19,74,', '2019-07-12,47,优,20,32,10,0.6,14,94,', '2019-07-13,77,良,20,44,10,0.6,16,132,', '2019-07-14,69,良,20,50,10,0.6,17,122,', '2019-07-15,64,良,24,48,10,0.6,18,116,', '2019-07-16,51,良,30,52,6,0.6,24,60,', '2019-07-17,29,优,18,29,6,0.6,22,42,', '2019-07-18,28,优,13,24,4,0.8,22,54,', '2019-07-19,41,优,14,32,6,0.6,15,82,', '2019-07-20,42,优,18,42,4,0.5,14,80,', '2019-07-21,47,优,22,47,5,0.6,14,89,', '2019-07-22,36,优,17,36,6,0.5,13,64,', '2019-07-23,35,优,12,33,9,0.4,16,70,', '2019-07-24,46,优,20,46,7,0.5,18,89,', '2019-07-25,52,良,21,46,7,0.6,15,102,', '2019-07-26,54,良,18,44,7,0.5